In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

# API Connection
from sodapy import Socrata

client = Socrata("data.cityofnewyork.us", "yUnYRTjXHWdK6ATxbXOsJR52v", timeout=60)

# First 20,000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("qgea-i56i", limit=20000)
df = pd.DataFrame.from_records(results)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   cmplnt_num                   20000 non-null  object
 1   cmplnt_fr_dt                 20000 non-null  object
 2   cmplnt_fr_tm                 20000 non-null  object
 3   cmplnt_to_dt                 18586 non-null  object
 4   cmplnt_to_tm                 20000 non-null  object
 5   addr_pct_cd                  19996 non-null  object
 6   rpt_dt                       20000 non-null  object
 7   ky_cd                        20000 non-null  object
 8   ofns_desc                    20000 non-null  object
 9   pd_cd                        19982 non-null  object
 10  pd_desc                      20000 non-null  object
 11  crm_atpt_cptd_cd             20000 non-null  object
 12  law_cat_cd                   20000 non-null  object
 13  boro_nm                      20

In [9]:
#view columns before dropping
df.columns

Index(['cmplnt_num', 'cmplnt_fr_dt', 'cmplnt_fr_tm', 'cmplnt_to_dt',
       'cmplnt_to_tm', 'addr_pct_cd', 'rpt_dt', 'ky_cd', 'ofns_desc', 'pd_cd',
       'pd_desc', 'crm_atpt_cptd_cd', 'law_cat_cd', 'boro_nm',
       'loc_of_occur_desc', 'prem_typ_desc', 'juris_desc', 'jurisdiction_code',
       'parks_nm', 'hadevelopt', 'housing_psa', 'x_coord_cd', 'y_coord_cd',
       'susp_age_group', 'susp_race', 'susp_sex', 'latitude', 'longitude',
       'lat_lon', 'patrol_boro', 'station_name', 'vic_age_group', 'vic_race',
       'vic_sex', ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer',
       ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q',
       ':@computed_region_sbqj_enih', 'transit_district'],
      dtype='object')

In [10]:
# Drop columns that are not needed
df.drop(columns=[":@computed_region_efsh_h5xi",
                  ":@computed_region_f5dn_yrer", 
                  ":@computed_region_yeji_bk3q", 
                  ":@computed_region_92fq_4b7q", 
                  ":@computed_region_sbqj_enih", 
                  "station_name",
                  "latitude",
                  "longitude",
                  "lat_lon",
                  'x_coord_cd',
                  'y_coord_cd',
                  'transit_district',
                  'pd_cd',
                  'ky_cd',
                  'cmplnt_num'])


,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ofns_desc,pd_desc,crm_atpt_cptd_cd,law_cat_cd,...,parks_nm,hadevelopt,housing_psa,susp_age_group,susp_race,susp_sex,patrol_boro,vic_age_group,vic_race,vic_sex
0,2022-12-31T00:00:00.000,11:25:00,2022-12-31T00:00:00.000,11:30:00,18,2022-12-31T00:00:00.000,CRIMINAL MISCHIEF & RELATED OF,"MISCHIEF,CRIMINAL, UNCL 2ND",COMPLETED,FELONY,...,(null),(null),(null),UNKNOWN,UNKNOWN,U,PATROL BORO MAN SOUTH,UNKNOWN,UNKNOWN,D
1,2022-12-31T00:00:00.000,09:18:00,2022-12-31T00:00:00.000,09:25:00,66,2022-12-31T00:00:00.000,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,...,(null),(null),(null),25-44,BLACK HISPANIC,M,PATROL BORO BKLYN SOUTH,45-64,WHITE HISPANIC,F
2,2022-12-31T00:00:00.000,01:55:00,2022-12-31T00:00:00.000,01:58:00,67,2022-12-31T00:00:00.000,VEHICLE AND TRAFFIC LAWS,LEAVING SCENE-ACCIDENT-PERSONA,COMPLETED,MISDEMEANOR,...,(null),(null),(null),UNKNOWN,UNKNOWN,U,PATROL BORO BKLYN SOUTH,25-44,BLACK,M
3,2022-12-30T00:00:00.000,23:30:00,2022-12-31T00:00:00.000,00:35:00,72,2022-12-31T00:00:00.000,ASSAULT 3 & RELATED OFFENSES,"MENACING,UNCLASSIFIED",COMPLETED,MISDEMEANOR,...,(null),(null),(null),45-64,BLACK HISPANIC,M,PATROL BORO BKLYN SOUTH,18-24,WHITE HISPANIC,M
4,2022-12-31T00:00:00.000,04:20:00,2022-12-31T00:00:00.000,04:25:00,6,2022-12-31T00:00:00.000,PETIT LARCENY,"LARCENY,PETIT FROM OPEN AREAS,",COMPLETED,MISDEMEANOR,...,(null),(null),(null),UNKNOWN,UNKNOWN,U,PATROL BORO MAN SOUTH,25-44,WHITE,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2022-11-11T00:00:00.000,11:05:00,2022-11-11T00:00:00.000,11:07:00,49,2022-12-16T00:00:00.000,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,...,(null),(null),(null),45-64,BLACK,M,PATROL BORO BRONX,UNKNOWN,UNKNOWN,D
19996,2022-12-16T00:00:00.000,18:50:00,NaN,(null),14,2022-12-16T00:00:00.000,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI",COMPLETED,MISDEMEANOR,...,(null),(null),(null),25-44,WHITE,M,PATROL BORO MAN SOUTH,UNKNOWN,UNKNOWN,E
19997,2022-12-03T00:00:00.000,20:59:00,2022-12-05T00:00:00.000,15:00:00,45,2022-12-16T00:00:00.000,THEFT-FRAUD,"FRAUD,UNCLASSIFIED-FELONY",COMPLETED,FELONY,...,(null),(null),(null),(null),(null),(null),PATROL BORO BRONX,25-44,WHITE HISPANIC,F
19998,2022-12-16T00:00:00.000,05:45:00,2022-12-16T00:00:00.000,06:00:00,32,2022-12-16T00:00:00.000,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,...,(null),(null),(null),25-44,BLACK,M,PATROL BORO MAN NORTH,25-44,WHITE HISPANIC,M
